In [1]:
import cobra.io
import pandas as pd
import math

In [2]:
# Load model
model = cobra.io.read_sbml_model('Roseobacter-litoralis-strain-B14.SBML')

In [3]:
# List of all exchange reactions in the medium except glucose
medium = model.medium
mediumwoglc = list(medium.keys())
mediumwoglc.remove('EX_glc__D_e')

In [4]:
# Initializing medium to what was found in the medium optimization file
for EX in mediumwoglc:
    medium[EX] = 2.5
    model.medium = medium

In [5]:
# List of medium with glucose
mediumwglc = list(medium.keys())

In [6]:
# Empty list to use in loop
csources = list()

In [7]:
# This loop finds all carbon source exchange reactions in the medium, and puts them 
# in the csources list
for reactionname in mediumwglc:
    reaction = model.reactions.get_by_id(reactionname)
    metabolite = reaction.reactants[0]
    comp = metabolite.formula
    checkcarbon = 'C' in comp
    if checkcarbon == True:
        csources.append(reaction.id)

In [8]:
# Some ions was wrongly identified in the loop, as they contain the letter 'C' in 
# their composition, but does not cotain carbon. They are removed from the list.
csources.remove('EX_cl_e')
csources.remove('EX_ca2_e')
csources.remove('EX_cobalt2_e')
csources.remove('EX_cu2_e')

In [9]:
# This loop removes all the carbon sources from the medium
medium1 = model.medium
for Csource in csources:
    medium1[Csource]=0
model.medium = medium1

In [10]:
# The carbon sources tested in the experiments are found by: 
# model.metabolites.query('Alanine','Name')    <- Ex for alanine
# The metabolites with the ending _e, contains the exchange reaction id under the 
# reactions they participate in.

In [11]:
# List of all the tested carbon sources. The model did not contain exchange reactions 
# for asparagine, citrulline, cysteine, phenylalanine tryptophan, tyrosine, creatine,
# creatinine, dimethylglycine, sacrosine and glycogen. This represent a shortcoming 
# of the model since some of the carbon sources should be able to promote growth of
# Rosebacter. Although, in the further analysis, this will be ignored.
newcsources = ['EX_ala__L_e','EX_arg__L_e','EX_asp__L_e', 'EX_glu__L_e', 'EX_gln__L_e', 
               'EX_gly_e', 'EX_his__L_e', 'EX_ile__L_e','EX_leu__L_e', 'EX_lys__L_e', 'EX_met__L_e', 
               'EX_orn_e', 'EX_pro__L_e', 'EX_ser__D_e', 'EX_thr__L_e', 'EX_val__L_e', 'EX_cellb_e', 
               'EX_arab__D_e', 'EX_arab__L_e', 'EX_fru_e', 'EX_fuc__L_e', 'EX_gal_e', 'EX_gam_e',
               'EX_glc__D_e', 'EX_lcts_e', 'EX_lac__D_e', 'EX_lac__L_e', 'EX_malt_e', 'EX_mnl_e', 
               'EX_man_e', 'EX_melib_e', 'EX_rmn_e', 'EX_rib__D_e','EX_sucr_e', 'EX_sbt__D_e', 
               'EX_tre_e', 'EX_xyl__D_e', 'EX_chol_e', 'EX_glyb_e', 'EX_ptrc_e', 'EX_taur_e', 
               'EX_etoh_e', 'EX_glyc_e', 'EX_urea_e']

Missing in newsources: asparagine, citrulline, cysteine, phenylalanine, tryptophan, tyrosine, creatine, creatinine, dimethylglycine, sarcosine, glycogen

In [12]:
# This loop introduces the tested carbon sources one at a time and checks for growth
# If the tested carbon source results in growth, meaning that the growth rate is a 
# non-zero value, a printed output is produced consisting of the tested carbon source
# and the growth rate obtained when using that carbon source.
for newcsource in newcsources:
    model.medium = medium1
    medium2 = model.medium
    medium2[newcsource]=10
    model.medium = medium2
    mumax = model.optimize().objective_value
    if mumax != 0.0:
        print(newcsource,mumax)

EX_arg__L_e 0.11832027929981208


In [13]:
# It is seen that there are only growth on Arginine. The most likely explanation
# for this is that there are not a appropriate nitrogen source present. The reason
# why growth on arginie is possible, is because aginine itself contains nitrogen,
# and can therefore act as the nitrogen source.

In [14]:
# To find out if any other nitogen sources can be added in the test for growth on
# the carbon sources, a list of all the nitrogensources in the model is produced.

In [15]:
# This loop finds all of the nitrogen sources in the model, and puts them in the
# list nsources.
allex = model.exchanges
nsources = list()
for reaction in allex:
    metabolite = reaction.reactants[0]
    comp = metabolite.formula
    checkcarbon = 'N' in comp
    if checkcarbon == True:
        nsources.append(reaction.id)

In [16]:
# This loop tests each nitrogen source for growth with glucose as the carbon source
# If a nitrogen source results in a non-zero growth rate, the name of the nitrogen-
# soruce exchange reaction is printed with the growth rate.
for nsource in nsources:
    model.medium=medium1
    medium3 = model.medium
    medium3['EX_glc__D_e']=10
    medium3[nsource] = 10
    model.medium = medium3
    gr = model.optimize().objective_value
    if gr != 0:
        print(gr,nsource)

0.24756662199071044 EX_arg__L_e


In [17]:
# It is seen that the only nitrogen source that results in growth is arginine,
# so this does not result in any nitrogen source that can be applied to the
# test for growth on carbon sources

In [18]:
# A new carveme is now used to produce a model of rosebacter litoralis on M9
# medium. This will create reactions to make growth on other nitrogen sources
# feasiable.

In [19]:
# The code producing the carveme was: 
#!carve --refseq GCF_014337925.1 --output Roseobacter-litoralis-strain-B14-M9.xml --gapfill M9 --init M9

In [20]:
# The model with M9 medium is loaded
M9model = cobra.io.read_sbml_model('Roseobacter-litoralis-strain-B14-M9.xml')

In [21]:
# The M9 medium composition
M9model.medium

{'EX_glc__D_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_h_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_k_e': 10.0,
 'EX_ca2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_cobalt2_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_o2_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mobd_e': 10.0,
 'EX_so4_e': 10.0}

In [22]:
# We now have a minimal medium where the only nitrogen source that is present
# is NH4. In this way, we can now test each carbon source without adding any
# other nitrogen source that also contains carbon (which then could also act
# as a carbon source, and therefore would result in growth no matter what
# carbon source is tested)

In [23]:
# The growth rate in the M9 medium
M9model.optimize().objective_value

0.7497699143341323

In [24]:
# Glucose is removed from the medium to be able to determine growth the tested
# carbon sources. 
medium = M9model.medium
medium['EX_glc__D_e'] = 0
M9model.medium = medium

In [25]:
# It is now again checked if all of the exchange reactions for the tested carbon
# sources is in the M9 model, and it is seen that all of those that were not
# present in the model before, is still not present. Furthermore, it was also
# found that the exchange reaction for urea was not present in this model either.

In [26]:
# Urea is removed from the list of carbon sources to be tested.
newcsources.remove('EX_urea_e')

Missing in newsources: asparagine, citrulline, cysteine, phenylalanine, tryptophan, tyrosine, creatine, creatinine, dimethylglycine, sarcosine, glycogen, urea

In [27]:
# This loop introduces the tested carbon sources one at a time and checks for growth
# If the tested carbon source results in growth, meaning that the growth rate is a 
# non-zero value, the interger 1 is put into the growthmodel list. If the tested
# carbon source does not lead to growth, the interger 0 is instead added to the
# growthmodel list.
growthmodel = list()
for newcsource in newcsources:
    M9model.medium = medium
    medium1 = M9model.medium
    medium1[newcsource]=10
    M9model.medium = medium1
    mumax = M9model.optimize().objective_value
    if mumax != 0.0:
        growthmodel.append(1)
    else:
        growthmodel.append(0)

In [28]:
# A data frame showing the carbon sources, and whether or not they lead to growth is
# produced. 1 = the carbon source leads to growth, 0 = the carbon source does not
# lead to growth.
csourcegrowthmodel = pd.DataFrame({'Carbon source': newcsources, 'Growth': growthmodel})
csourcegrowthmodel

,Carbon source,Growth
0,EX_ala__L_e,1
1,EX_arg__L_e,1
2,EX_asp__L_e,1
3,EX_glu__L_e,1
4,EX_gln__L_e,1
5,EX_gly_e,1
6,EX_his__L_e,1
7,EX_ile__L_e,0
8,EX_leu__L_e,0
9,EX_lys__L_e,1


In [29]:
# The results from the litterature is put into the growthexp list. Again where
# 1 = carbon source leads to growth, and 0 = carbon source does not lead to growth.
growthexp = [1,1,1,0,1,1,1,0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,0,0,1,1,0,1,1,1,1,1,0,0,0,1,1,1,1,0,1,1]

In [30]:
# A data frame showing the carbon sources, and whether or not they lead to growth in
# the experiment is produced.
csourcegrowthexp = pd.DataFrame({'Carbon source': newcsources, 'Growth': growthexp})
csourcegrowthexp

,Carbon source,Growth
0,EX_ala__L_e,1
1,EX_arg__L_e,1
2,EX_asp__L_e,1
3,EX_glu__L_e,0
4,EX_gln__L_e,1
5,EX_gly_e,1
6,EX_his__L_e,1
7,EX_ile__L_e,0
8,EX_leu__L_e,0
9,EX_lys__L_e,0


In [68]:
for i in range(len(growthexp)):
    if growthexp[i] == 1:
        growthexp[i] = 'Yes'
    if growthexp[i] == 0:
        growthexp[i] = 'No'
    if growthmodel[i] == 1:
        growthmodel[i] = 'Yes'
    if growthmodel[i] == 0:
        growthmodel[i] = 'No'
for i in range(len(newcsources)):
    reaction = newcsources[i]
    meta = model.reactions.get_by_id(reaction).reactants[0]
    newcsources[i] = meta.name
growthcsources = pd.DataFrame({'Carbon source': newcsources, 'Growth in experiment': growthexp, 'Growth in model':growthmodel})
growthcsources.set_index('Carbon source', inplace=True)
growthcsources

,Growth in experiment,Growth in model
Carbon source,,
L-Alanine,Yes,Yes
L-Arginine,Yes,Yes
L-Aspartate,Yes,Yes
L-Glutamate,No,Yes
L-Glutamine,Yes,Yes
Glycine,Yes,Yes
L-Histidine,Yes,Yes
L-Isoleucine,No,No
L-Leucine,No,No


In [69]:
growthcsources.to_csv('growthcsources.csv')

In [42]:
TP = 0
FP = 0
TN = 0
FN = 0
for i in range(42):
    exp = csourcegrowthexp.iloc[i,1]
    mod = csourcegrowthmodel.iloc[i,1]
    if exp == 1:
        if mod == 1:
            TP = TP + 1
        elif mod == 0:
            FN = FN + 1
    if exp == 0:
        if mod == 0:
            TN = TN + 1
        elif mod == 1:
            FP = FP +1

In [44]:
print(TP)
print(TN)
print(FP)
print(FN)

23
6
10
3


In [41]:
range(42)

range(0, 42)

In [50]:
Precision = TP/(TP+FP) #Positive predictive value

In [51]:
Precision #From 0 to 1, 1 being perfect prediction

0.696969696969697

In [58]:
NPV = TN/(TN+FN)

In [59]:
NPV

0.6666666666666666

In [60]:
Sensitivity = TP/(TP+FN)

In [61]:
Sensitivity

0.8846153846153846

In [62]:
Specificity = TN/(TN+FP)

In [63]:
Specificity

0.375

In [64]:
Accuracy = (TP+TN)/(TP+TN+FP+FN)

In [65]:
Accuracy

0.6904761904761905

In [66]:
F1score = 2*Precision*Sensitivity/(Precision+Sensitivity)

In [67]:
F1score #From 0 to 1, 1 being perfect prediction

0.7796610169491526

In [68]:
MCC = (TP*TN-FP*FN)/(math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)))

In [69]:
MCC  #from -1 to 1, where 0 would be random guessing

0.3072549338995135